In [1]:
import numpy as np
import math

# Utility functions

In [2]:
def world2local(x, points):
    '''
    Arguments:
        x: state vectors (= [[x, y, theta], ...], shape of [n, 3])
        points: positions in world coordinate (= [[x1, y1], [x2, y2], ...], shape of [m, 2])
    Return:
        positions in local coordinates ( = [[[x1, y1], [x2, y2], ...], ...], shape of [n, m, 2])
    '''
    
    n = x.shape[0]
    m = points.shape[0]
    
    theta = x[:, 2] # shape of [n,]
    
    points = points[np.newaxis, :, :]     # shape of [1, m, 2]
    points = np.repeat(points, n, axis=0) # shape of [n, m, 2]
    x = x[:, np.newaxis, :2]              # shape of [n, 1, 2]
    d = points - x                        # shape of [n, m, 2]
    d = d[:, :, :, np.newaxis]            # shape of [n, m, 2, 1]
    
    cos, sin = np.cos(theta), np.sin(theta) # shape of [n,]
    rot = np.array([
        [ cos, sin],
        [-sin, cos]
    ])                               # shape of [2, 2, n]
    rot = rot.transpose(2, 0, 1)     # shape of [n, 2, 2]
    rot = rot[:, np.newaxis, :, :]   # shape of [n, 1, 2, 2]
    rot = np.repeat(rot, m, axis=1)  # shape of [n, m, 2, 2]
    
    points_local = np.matmul(rot, d)        # shape of [n, m, 2, 1]
    points_local = points_local[:, :, :, 0] # shape of [n, m, 2]
    
    return points_local

# State transition model

In [3]:
def f(x, u, w, dt):
    '''
    Arguments:
        x: state vectors (= [[x, y, theta], ...], shape of [n, 3])
        u: control input vector (= [v, omega], shape of [2,])
        w: control error vectors (= [[w_v, w_omega, gamma], ...], shape of [n, 3])
        dt: integral interval (scalar)
    Return:
        updated state vector (same shape w/ x)
    '''
    
    theta = x[:, 2]
    
    v = u[0] + w[:, 0]
    omega = u[1] + w[:, 1]
    gamma = w[:, 2]
    
    r = v / omega
    
    dx = -r * np.sin(theta) + r * np.sin(theta + omega * dt)
    dy =  r * np.cos(theta) - r * np.cos(theta + omega * dt)
    dtheta = omega * dt + gamma * dt
    
    delta = np.array([
        dx,
        dy,
        dtheta
    ]).T
    
    x_new = x + delta
    
    return x_new

# Observation model

In [4]:
landmarks = np.array([
    [ 5.0,  5.0],
    [ 2.0, -3.0],
    [-3.0,  4.0],
    [-5.0, -1.0],
    [ 0.0,  0.0]
])

def g(x, v):
    '''
    Arguments:
        x: state vectors (= [[x, y, theta], ...], shape of [n, 3])
        v: observation error vectors ( = [[[x1, y1], [x2, y2], ...], ...], shape of [n, m, 2])
    Return:
        landmark positions observed from local coordinates (same shape w/ v)
    '''
    
    z = world2local(x, landmarks) + v
    
    return z